# Project 2

## Part 1: Reddit (HTML to MongoDB)

In [1]:
import re
import requests
from bs4 import BeautifulSoup as bs
from pymongo import MongoClient

In [2]:
# Source Reddit
posts = []
url = "http://reddit.com"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.122 Safari/537.36'}
html = requests.get(url, headers=headers).text
parser = bs(html)

In [3]:
# Destination MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client.reddit
db.posts.drop()
collection = db.posts

In [4]:
# Find comments <span> element; it is the only elements all posts have in common
comments = parser.find_all("span", text=re.compile(".*comments"))

# Get the parent post from the comment and scrape the data
for comment in comments:
    post = comment.parent.parent.parent.parent
    # skip sponsored  or pinned content
    if len(post.find_all("span", text="promoted")) > 0 \
        or len(post.find_all("span", text="pinned by moderators")):
        continue
    # get post data
    post_data = {}
    post_data["title"] = post.find_all("h3")[0].text
    post_data["link"] = post.find_all("h3")[0].parent.parent["href"]
    post_data["age"] =" ".join(post.find_all("a", text=re.compile(".*ago"))[0].text.split(" ")[0:2])
    post_data["comments"] = post.find_all("span", text=re.compile(".*comments"))[0].text.split(" ")[0]
    post_data["subreddit"] = "Main Page / " + post.find_all("h3")[0].parent.parent["href"].split('/')[2]
    posts.append(post_data)

# Let's print evidence that something is working
for post in posts:
    print("Title: " + post["title"][:80])

Title: What was the very first movie to make you cry?
Title: Wild Karen loses mind over car on crosswalk
Title: Texas park ranger pushed into water after reminding crowd about social distancin
Title: I made a big flip book during quarantine. My love to everyone who is struggling 
Title: I made a really big flip book during quarantine and people said to post it here.
Title: A guide to mattress dimensions and bed sizes
Title: One Death Is a Tragedy. 60,000 Deaths Are a Great Success. Most presidents try t


In [5]:
# Let's also scrape some of the more interesting subreddits
subreddits = ["science", "technology", "programming", "AskReddit", "news"]

# Get the parent post from the comment and scrape the data
for reddit in subreddits:
    # go to subreddit
    url = "http://reddit.com/r/" + reddit
    html = requests.get(url, headers=headers).text
    parser = bs(html)
    # get comments <span>
    comments = parser.find_all("span", text=re.compile(".*comments"))
    # get parent post and scrape data
    for comment in comments:
        post = comment.parent.parent.parent.parent
        # skip sponsored or pinned content
        if len(post.find_all("span", text="promoted")) > 0 \
            or len(post.find_all("span", text="pinned by moderators")):
            continue
        # get post data
        post_data = {}
        post_data["title"] = post.find_all("h3")[0].text
        post_data["link"] = post.find_all("h3")[0].parent.parent["href"]
        post_data["age"] =" ".join(post.find_all("a", text=re.compile(".*ago"))[0].text.split(" ")[0:2])
        post_data["comments"] = post.find_all("span", text=re.compile(".*comments"))[0].text.split(" ")[0]
        post_data["subreddit"] = post.find_all("h3")[0].parent.parent["href"].split('/')[2]
        posts.append(post_data)

print("Total posts collected: " + str(len(posts)))

Total posts collected: 37


In [6]:
# Put all collected posts into MongoDB
for post in posts:
    collection.insert_one(post)

In [7]:
# Let print the titles, since that's all anyone reads anyways
x = collection.find()
for z in x:
    print(z["subreddit"] + "\r\n" + z["title"] + "\r\n")

Main Page / AskReddit
What was the very first movie to make you cry?

Main Page / PublicFreakout
Wild Karen loses mind over car on crosswalk

Main Page / news
Texas park ranger pushed into water after reminding crowd about social distancing

Main Page / woahdude
I made a big flip book during quarantine. My love to everyone who is struggling right now!

Main Page / nextfuckinglevel
I made a really big flip book during quarantine and people said to post it here. My love to everyone who is struggling right now!

Main Page / coolguides
A guide to mattress dimensions and bed sizes

Main Page / politics
One Death Is a Tragedy. 60,000 Deaths Are a Great Success. Most presidents try to console the nation in moments of grief, but Donald Trump is taking a victory lap.

science
Green method could enable hospitals to produce hydrogen peroxide in house. A team of researchers has developed a portable, more environmentally friendly method to produce hydrogen peroxide. It could enable hospitals to mak

In [8]:
# Close database connection
client.close()

# Part 2: Iris Flower Dataset (API to SQL)

In [9]:
from sklearn import datasets
import numpy as np

In [10]:
# Source scikit-learn
iris = datasets.load_iris()

In [11]:
# What is the Iris database?
print("Description" + "\n".join(iris.DESCR.split("\n")[35:46]))

Description
The famous Iris database, first used by Sir R.A. Fisher. The dataset is taken
from Fisher's paper. Note that it's the same as in R, but not as in the UCI
Machine Learning Repository, which has two wrong data points.

This is perhaps the best known database to be found in the
pattern recognition literature.  Fisher's paper is a classic in the field and
is referenced frequently to this day.  (See Duda & Hart, for example.)  The
data set contains 3 classes of 50 instances each, where each class refers to a
type of iris plant.  One class is linearly separable from the other 2; the
latter are NOT linearly separable from each other.


In [12]:
# What data does it contain?
print(iris.feature_names)
print(iris.data[0:5])

['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]


In [13]:
# Each row corresponds to an iris flower species
print(iris.target_names)
print(list(iris.target))

['setosa' 'versicolor' 'virginica']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [14]:
# Aggregate the data
header = np.append(iris.feature_names, "target")
table = np.append(iris.data, [[iris.target_names[target]] for target in iris.target], axis=1)

In [15]:
# Destination SQL Script
f = open("iris.sql", "w")

# Create table
print("DROP TABLE IF EXISTS IRIS;")
print("CREATE TABLE IRIS (")
print("  SEPAL_LENGTH FLOAT,")
print("  SEPAL_WIDTH FLOAT,")
print("  PETAL_LENGTH FLOAT,")
print("  PETAL_WIDTH FLOAT,")
print("  TARGET ENUM(" + str(list(iris.target_names))[1:-1] + ")")
print(");")

f.write("DROP TABLE IF EXISTS IRIS;\n")
f.write("CREATE TABLE IRIS (\n")
f.write("  SEPAL_LENGTH FLOAT,\n")
f.write("  SEPAL_WIDTH FLOAT,\n")
f.write("  PETAL_LENGTH FLOAT,\n")
f.write("  PETAL_WIDTH FLOAT,\n")
f.write("  TARGET ENUM(" + str(list(iris.target_names))[1:-1] + ")\n")
f.write(");\n")

print()
f.write("\n")

# Insert data
for row in table[0:5]:
    print("INSERT INTO IRIS (SEPAL_LENGTH, SEPAL_WIDTH, PETAL_LENGTH, PETAL_WIDTH, TARGET)")
    print("  VALUES ('" + "', '".join(row) + "');")

for row in table:
    f.write("INSERT INTO IRIS (SEPAL_LENGTH, SEPAL_WIDTH, PETAL_LENGTH, PETAL_WIDTH, TARGET)\n")
    f.write("  VALUES ('" + "', '".join(row) + "');\n")

# Close SQL Script
f.close()

DROP TABLE IF EXISTS IRIS;
CREATE TABLE IRIS (
  SEPAL_LENGTH FLOAT,
  SEPAL_WIDTH FLOAT,
  PETAL_LENGTH FLOAT,
  PETAL_WIDTH FLOAT,
  TARGET ENUM('setosa', 'versicolor', 'virginica')
);

INSERT INTO IRIS (SEPAL_LENGTH, SEPAL_WIDTH, PETAL_LENGTH, PETAL_WIDTH, TARGET)
  VALUES ('5.1', '3.5', '1.4', '0.2', 'setosa');
INSERT INTO IRIS (SEPAL_LENGTH, SEPAL_WIDTH, PETAL_LENGTH, PETAL_WIDTH, TARGET)
  VALUES ('4.9', '3.0', '1.4', '0.2', 'setosa');
INSERT INTO IRIS (SEPAL_LENGTH, SEPAL_WIDTH, PETAL_LENGTH, PETAL_WIDTH, TARGET)
  VALUES ('4.7', '3.2', '1.3', '0.2', 'setosa');
INSERT INTO IRIS (SEPAL_LENGTH, SEPAL_WIDTH, PETAL_LENGTH, PETAL_WIDTH, TARGET)
  VALUES ('4.6', '3.1', '1.5', '0.2', 'setosa');
INSERT INTO IRIS (SEPAL_LENGTH, SEPAL_WIDTH, PETAL_LENGTH, PETAL_WIDTH, TARGET)
  VALUES ('5.0', '3.6', '1.4', '0.2', 'setosa');
